In [13]:
import argparse
import os
from copy import deepcopy
from typing import Optional, Tuple

import gymnasium as gym
import numpy as np
import torch
from pettingzoo.classic import hanabi_v4
from torch.utils.tensorboard import SummaryWriter

from tianshou.data import Collector, PrioritizedVectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import (
    BasePolicy,
    RainbowPolicy,
    MultiAgentPolicyManager,
    RandomPolicy,
)
from tianshou.utils.net.discrete import NoisyLinear
from tianshou.trainer import offpolicy_trainer
from tianshou.utils import TensorboardLogger
from tianshou.utils.net.common import Net
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [14]:
p = {
    'hidden_layers': [256,256],
    'gamma': 0.99,
    'lr': 1e-6,
    'target_update_freq': 200,
    'estimation_steps': 1,
    'num_train':50,
    'num_test':50,
    'buffer_size':50000,
    'batch_size':32,
    'steps_per_collect': 10000,
    'updates_per_train': 500,
    'test_steps': 50000,
    'epochs':1000,
    'test_frequency':5,
    'save_frequency':25,
    'eps':0.01
}
path = 'saved_data/training_group_2/'

In [15]:
def get_env(render_mode=None):
    return PettingZooEnv(hanabi_v4.env(colors=2, ranks=5, players=2, hand_size=4, max_information_tokens=5,
max_life_tokens=2, observation_type=1))

In [16]:
def get_agents(
    lr = 1e-4, 
    hidden_layers = [256, 256], 
    gamma = 0.99,
    target_update_freq = 200, 
    estimation_steps = 1, 
    num_train = 50, 
    num_test = 50,
    atom_size = 51,
    noisy_std = 0.1
):
    
    def noisy_linear(x, y):
        return NoisyLinear(x, y, noisy_std)
    
    # Return Policy, Agents, Envs
    env = get_env()
    observation_space = env.observation_space['observation'] if isinstance(
    env.observation_space, gym.spaces.Dict
    ) else env.observation_space

    state_shape = observation_space.shape or observation_space.n
    action_shape = env.action_space.shape or env.action_space.n
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    net1 = Net(
            state_shape,
            action_shape,
            hidden_sizes=hidden_layers,
            device = device,
            softmax = True,
            num_atoms = atom_size,
            dueling_param = ({
                'linear_layer': noisy_linear
            }, {
                'linear_layer': noisy_linear})
    )

    net2 = Net(
            state_shape,
            action_shape,
            hidden_sizes=hidden_layers,
            device = device,
            softmax = True,
            num_atoms = atom_size,
            dueling_param = ({
                'linear_layer': noisy_linear
            }, {
                'linear_layer': noisy_linear})
    )

    optim1 = torch.optim.Adam(net1.parameters(), lr= lr)
    optim2 = torch.optim.Adam(net2.parameters(), lr = lr)

    agent1 = RainbowPolicy(
            net1,
            optim1,
            gamma,
            num_atoms = atom_size,
            v_min = -10,
            v_max = 10,
            estimation_step = estimation_steps,
            target_update_freq=target_update_freq
        ).to(device)

    agent2 = RainbowPolicy(
            net2,
            optim2,
            gamma,
            num_atoms = atom_size,
            v_min = -10,
            v_max = 10,
            estimation_step = estimation_steps,
            target_update_freq=target_update_freq
        ).to(device)

    agents = [agent1, agent1]
    policy = MultiAgentPolicyManager(agents, env)
    agents = env.agents

    train_envs = DummyVectorEnv([get_env for _ in range(num_train)])
    test_envs = DummyVectorEnv([get_env for _ in range(num_test)])
    
    return policy, agents, train_envs, test_envs

In [17]:
def get_collectors(
    policy,
    train_envs,
    test_envs,
    buffer_size
):
    
    # Get collectors
    train_collector = Collector(
    policy,
    train_envs,
    PrioritizedVectorReplayBuffer(buffer_size, len(train_envs), alpha = 0.6, beta = 0.4, weight_norm=True),
    exploration_noise=True)
    
    test_collector = Collector(policy, test_envs, exploration_noise=True)
    
    return train_collector, test_collector

In [18]:
def initialize_buffer(
    train_collector,
    buffer_size,
    agents,
    policy,
    eps = 0.1
):
    for a in agents:
        policy.policies[a].set_eps(1)
    train_collector.collect(n_step = buffer_size)
    for a in agents:
        policy.policies[a].set_eps(eps)

In [19]:
def save_policy(policy, agents):
    for a in agents:
        torch.save(policy.policies[a].state_dict(), f'saved_data/training_group_2/{a}_params.pth')

def save_history(history):
    np.save(f'saved_data/training_group_2/training_rewards.npy', np.array(history))
    
def change_lr(optimizer, new_lr):
    # Run this to change the learning rate to 1e-5:
    for g in optimizer.param_groups:
        g['lr'] = new_lr

In [1]:
def train(
    policy,
    train_collector,
    test_collector,
    agents,
    epochs=1000,
    collection_steps_per_epoch=10000,
    updates_per_epoch=500,
    test_frequency=5,
    test_steps=50000,
    save_frequency = 50,
    batch_size = 32,
    eps = 0.1,
    training_history = []
):
    for i in tqdm(range(epochs)):
        
        # Collection step
        result = train_collector.collect(n_step = collection_steps_per_epoch)
        
        # Test Step
        if i%test_frequency == 0:
            for a in agents:
                policy.policies[a].set_eps(0)
            result = test_collector.collect(n_step = test_steps)
            mean_reward = result['rews'].mean()
            tqdm.write(str(mean_reward))
            training_history.append(mean_reward)
            for a in agents:
                policy.policies[a].set_eps(eps)
    
        if i%save_frequency == 0:
            save_policy(policy, agents)
            save_history(training_history)
    
        # Update step (one epoch)
        for _ in range(updates_per_epoch): 
            losses = policy.update(batch_size, train_collector.buffer)
    
    plot_and_save(training_history, test_frequency)

In [21]:
def plot_and_save(training_history, test_frequency, save = True):
    x = np.arange(len(training_history))
    x *= test_frequency
    plt.plot(x, training_history)
    plt.title('Combined Average Score (Rainbow, 2 Color game)')
    plt.xlabel('Epoch')
    plt.ylabel('Average Score (max 10)')
    if save: plt.savefig(f'saved_data/training_group_2/training_curve.png')
        
def load(policy, path, agents):
    for a in agents:
        policy.policies[a].load_state_dict(torch.load(path + f'{a}_params.pth'))
    his = list(np.load(path + f'training_rewards.npy'))
    return his

In [22]:
policy, agents, train_envs, test_envs = get_agents(lr = p['lr'], hidden_layers = p['hidden_layers'], gamma = p['gamma'], 
    target_update_freq = p['target_update_freq'], estimation_steps = p['estimation_steps'], num_train = p['num_train'], 
    num_test = p['num_test'])
train_collector, test_collector = get_collectors(policy, train_envs, test_envs, buffer_size = p['buffer_size'])
initialize_buffer(train_collector, p['buffer_size'], agents, policy, eps = p['eps'])

In [23]:
training_history = load(policy, path, agents)

In [ ]:
train(policy, train_collector, test_collector, agents,
    epochs=p['epochs'],
    collection_steps_per_epoch=p['steps_per_collect'],
    updates_per_epoch=p['updates_per_train'],
    test_frequency=p['test_frequency'],
    test_steps=p['test_steps'],
    save_frequency = p['save_frequency'],
    batch_size = p['batch_size'],
    eps = p['eps'],
    training_history = training_history)

  0%|          | 0/1000 [00:00<?, ?it/s]

8.054115507048659
8.042476276547672
7.999548532731377
8.048670572329879
8.079350766456267
8.099188458070333
8.124943617501128
8.032897701667418
8.055405405405406
8.05507900677201
8.083258325832583
8.163292847503374
8.128262826282628
8.103386004514674
8.064269662921348
8.086642599277978
8.131922557406574
8.102390617952187
8.05103884372177
8.10521582733813
8.074492099322798
8.165693758419398
8.11971197119712
8.099412562132851
8.113758992805755
8.056884875846501
8.091891891891892
8.117303370786518
8.085662759242561
8.109353818346136
8.10153429602888
8.124044943820225
8.163292847503374
8.081227436823104
8.105808194506979
8.08228417266187
8.097968397291197
8.049616599007669
8.046825754164791
8.066335740072201
8.115523465703971
8.110661268556006
8.10158013544018
8.119208277103015
8.128654970760234
8.071139126519586
8.074424898511502
8.090418353576249
8.074224021592443
8.110207768744354
8.128205128205128
8.105168539325843
8.115713642503376
8.105026929982047
8.147629796839729
8.05668016194332
